##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Search Grounding

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Search_Grounding.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

This notebook contains a walkthrough of how to use the search grounding feature in the Gemini API.

In [1]:
import google.generativeai as genai
from IPython.display import Markdown, HTML

/home/m4s1t4/anaconda3/envs/Agents/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [4]:
import os
import dotenv
dotenv.load_dotenv()

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

## Search Grounding

Take a look at the available models from the Gemini API. To perform search grounding, use `'models/gemini-1.5-pro-002'` when calling the API.

In [5]:
for model in genai.list_models():
  if '002' in model.name:
    print(model.name)

models/gemini-1.5-pro-002
models/gemini-1.5-flash-002


`GoogleSearchRetrieval` is a tool to retrieve public web data for grounding, powered by Google. Part of `GoogleSearchRetrieval` is dynamic search retrieval, which uses real time retrieved data to build answers to queries.

Two possible ways to use include `GoogleSearchRetrieval` are to pass in a string or dictionary to the field. First, you will pass in `google_search_retrieval` to the `tools` parameter in `generate_content`.

In [9]:
model = genai.GenerativeModel('models/gemini-1.5-flash')
response = model.generate_content(contents="What is the land area of Spain?",
                                  tools='google_search_retrieval')

Take a look at the first response produced by the model.

In [10]:
Markdown(response.candidates[0].content.parts[0].text)

The land area of Spain is reported differently by various sources.  The discrepancy likely stems from different methodologies in including or excluding islands, and potentially from variations in data collection and updates.

Here's a summary of the reported figures:

* **Wikipedia [Source 1]:** 504,782 square kilometers (mainland) and 505,370 square kilometers (including islands).  Note that this source also mentions a land area of 499,542 square kilometers, but this is stated as part of the total area including water.

* **La Moncloa [Source 2]:** 506,030 square kilometers.  This source breaks down the area into mainland, Balearic Islands, Canary Islands, and Ceuta and Melilla.

* **Biodiversity Information System for Europe [Source 3]:** Approximately 506,000 square kilometers.

* **Trading Economics [Source 4]:** 499,733 square kilometers (data from 2021).

* **Spain.info [Source 5]:** Just over 505,000 square kilometers, with 12,500 square kilometers attributed to islands.


Therefore, while there's no single definitive answer, the most frequently cited figure is around **505,000 to 506,000 square kilometers**.  The variation is likely due to ongoing surveys and adjustments to data.


Now, take a look at the rendered HTML content gathered by the Gemini API.

> Important: If you use search grounding you **must** follow the [requirements outlined here](https://ai.google.dev/gemini-api/docs/grounding/search-suggestions?hl=en#requirements), which includes "Display the Search Suggestion exactly as provided" and "Take users directly to the Google Search results page (SRP) when they interact with the Search Suggestion".

In [11]:
HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

You will also receive links that direct you to sources used for your query.

In [12]:
response.candidates[0].grounding_metadata.grounding_chunks

[web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQ2TeiR_9Y4pyvIIHLRpcYu8o_m8Yae2--R1Y1_Yd6DzzwHLcjCG4cc_Q-NN4CtbbwLrbWzAHkxN_v05oPJgwi8QMXxfPJ6Bv6-lZf_xKJ9GbNrAue2KXwezigUEsws0LALpu1mWeQeGVjV99k7tE0vF-s41Xx9CiExsgGAZZb1TiLa24B1XAbvW4TDSSG-SphXQzcF"
  title: "lamoncloa.gob.es"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQ-PC3t3KJY03mWHXHRw9IjXShJy5sjCgHSREXTcaDzyfJWj6unx1WbPj-TEwfY6aWc74h679rVQZqoIFvEaOXWXaJJgsuAHFf7yU-mXsu1LXuI8uKiYHwXhqNUFphHKWPIE_4af7eGdgCnPjL4bpRB1D9EXsYpFejV"
  title: "tradingeconomics.com"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AYygrcQWvf-EMLUoiV8kkyDlGyIVJx6ICXU2kKKvCUMJ8wOVtyfxKHQw8NYdmfya6vqH9nieu69B92tYW1PjEcK2A2NWtdaZrl6IAlsDkPagL66ZmpZUJ487YoQrxxCwn0QYxpt0lHm9WpMFJBgyzqe0z7tI0Bccra-BHIW6mSAp7-_1CE8pug=="
  title: "spain.info"
}
]

And supporting segments, showing which parts of the output are quored from which source.

In [13]:
response.candidates[0].grounding_metadata.grounding_supports

[segment {
  start_index: 527
  end_index: 582
  text: "* **La Moncloa [Source 2]:** 506,030 square kilometers."
}
grounding_chunk_indices: 0
confidence_scores: 0.642883539
, segment {
  start_index: 584
  end_index: 688
  text: "This source breaks down the area into mainland, Balearic Islands, Canary Islands, and Ceuta and Melilla."
}
grounding_chunk_indices: 0
confidence_scores: 0.653900504
, segment {
  start_index: 793
  end_index: 872
  text: "* **Trading Economics [Source 4]:** 499,733 square kilometers (data from 2021)."
}
grounding_chunk_indices: 1
confidence_scores: 0.779980421
, segment {
  start_index: 874
  end_index: 992
  text: "* **Spain.info [Source 5]:** Just over 505,000 square kilometers, with 12,500 square kilometers attributed to islands."
}
grounding_chunk_indices: 2
confidence_scores: 0.736297548
]

### Customize search grounding parameters

For a dictionary implementation, you don't actually have to pass in `mode` or `dynamic_threshold` key-value pairs. If you don't pass in a value, a default value is created. See the next code cell for this. For the `mode` value, you can pass in one of two specifications:

1. `unspecified`: Always trigger retrieval.
2. `dynamic`: Run retrieval only when system decides it is necessary.

The `dynamic_threshold` is a float value that acts as a threshold on how relevant answers returned should be. You can tweak this parameter depending on how much variation you want in your answer.

In [14]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="What is the land area of Spain?",
                                  tools={"google_search_retrieval": {"dynamic_retrieval_config": {"mode": "MODE_DYNAMIC",
                                                                                                  "dynamic_threshold": 0.5}}})

In [15]:
Markdown(response.candidates[0].content.parts[0].text)

Spain's land area is approximately 500,000 square kilometers.  More precisely, different sources provide slightly different figures. For instance, Wikipedia mentions 504,782 square kilometers, while the World Bank data from 2021 indicates 499,733 square kilometers.  Including the Spanish island territories, the total area increases to around 505,000-506,000 square kilometers.

Spain is one of the fifty largest countries globally. It's the largest in Southern Europe, second largest in Western Europe (after France), and fourth largest on the European continent (after Russia, Ukraine, and France).

The country is located in Southwestern Europe, bordering the Mediterranean Sea, North Atlantic Ocean, Bay of Biscay, and Pyrenees Mountains.  It shares land borders with France, Andorra, Portugal, Gibraltar, and Morocco (around the cities of Ceuta and Melilla).  Spain's geography is dominated by a large central plateau, divided by mountain ranges, contributing to its average altitude of 650 meters.  This plateau is bordered by coastal plains and several important rivers.

Spain also possesses several island territories, including the Balearic Islands in the Mediterranean Sea and the Canary Islands in the Atlantic Ocean. Additionally, Spain controls two autonomous cities on the North African coast – Ceuta and Melilla – and a few small, uninhabited enclaves off the Moroccan coast.

It's worth noting that these figures might vary slightly depending on the source and measurement methods. Also, geopolitical changes can affect a country's area, though such changes are rare.


You can also pass in an empty dictionary. Default values for `mode` and `dynamic_threshold` will be used.

In [16]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="What is the land area of Spain?",
                                  tools={"google_search_retrieval": {}})

In [17]:
Markdown(response.candidates[0].content.parts[0].text)

Spain's land area is approximately 500,000 square kilometers.  More precisely, different sources provide slightly different figures, ranging from 499,542 sq km to 506,030 sq km. This variation is likely due to different inclusions of island territories and autonomous cities like Ceuta and Melilla.  The most commonly cited figure for the mainland is around 504,782 square kilometers.  Including the island territories increases this to approximately 505,370 sq km.

Spain is the largest country in Southern Europe, the second largest in Western Europe (after France), and the fourth largest on the European continent (after Russia, Ukraine, and France).  It's slightly more than twice the size of Oregon and almost five times the size of Kentucky.  The country features a diverse geography including a large central plateau (Meseta Central) surrounded by mountain ranges like the Pyrenees, along with coastal plains and island archipelagos. The average altitude is approximately 650 meters.  Spain shares land borders with France, Andorra, Portugal, Gibraltar, and Morocco (at the cities of Ceuta and Melilla). This answer is based on information available as of today, December 20, 2024, and might change in the future.


## With chat

If you attach the `google_search_retrieval` tool to the model, it can be used in every turn of the chat:

In [18]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002', tools="google_search_retrieval")
chat = model.start_chat()
result = chat.send_message('what is the area of spain?')

In [19]:
display(Markdown(result.text))

display(HTML(result.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Spain has a total area of approximately 505,370 square kilometers (195,124 square miles). This includes the Spanish mainland, the Balearic Islands located in the Mediterranean Sea, and the Canary Islands situated in the Atlantic Ocean.  The land area itself is slightly smaller, around 500,000 square kilometers, with the remaining area accounting for water bodies.
Spain is the largest country in Southern Europe, the second largest in Western Europe (after France), and the fourth largest on the European continent (after Russia, Ukraine, and France).


It's worth noting that these figures can vary slightly depending on the source and measurement methods used.  Also, Spain possesses two autonomous cities in North Africa, Ceuta and Melilla, and three small possessions off the coast of Morocco – Islas Chafarinas, Peñón de Alhucemas, and Peñón de Vélez de la Gomera, which are sometimes included when calculating the total area.


In [20]:
result = chat.send_message('what is the google stock price?')


display(Markdown(result.text))

display(HTML(result.candidates[0].grounding_metadata.search_entry_point.rendered_content))

As of December 20, 2024, Google's stock prices vary slightly depending on the stock class and the reporting source.

* **GOOG (Class C):** TradingView reports a price of $189.70, up 0.46% over the past 24 hours.

* **GOOGL (Class A):**  Several sources report prices around $188.40 - $188.47.  Angel One specifically lists $188.47.  Nasdaq data from earlier in the day indicates a range of $189.33 to $193.03.

* **GOOG (Class A):** Business Insider listed yesterday's (December 19, 2024) closing price as $188.43.

Please note that stock prices can fluctuate throughout the day, so these values are snapshots in time. For the most up-to-the-minute data, consult a live stock ticker.  Also, be sure to distinguish between the different classes of Google stock (Class A, GOOGL and Class C, GOOG) when making investment decisions, as they have different voting rights.


But you can also pass the tool through the `send_message` call.

In [22]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
chat = model.start_chat()
result = chat.send_message('what is the area of spain?', tools="google_search_retrieval")
result

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Spain covers a total area of approximately 505,370 square kilometers (195,124 square miles).  This includes the Spanish mainland, the Balearic Islands located in the Mediterranean Sea, and the Canary Islands situated in the Atlantic Ocean.  It's worth noting that two autonomous cities, Ceuta and Melilla, located in North Africa, are also part of Spain, along with a few small possessions off the coast of Morocco. The land area alone is slightly smaller, around 500,000 square kilometers, with the remaining area being water.  To put this into perspective, Spain is slightly more than twice the size of Oregon in the United States.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "grounding_met